In [419]:
import pandas as pd
import numpy as np
from src.make_preference_matrix import make_preference_df
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [420]:
df=make_preference_df()

In [3]:
df.head()

,_id,game_id,score,user_id,game_number,user_number
0,5c09771d933dea68aaeebb68,Grand Theft Auto V,8,J4MESOX4D,698,18776
1,5c09771d933dea68aaeebb69,Grand Theft Auto V,7,CardinalStorm,698,2369
2,5c09771d933dea68aaeebb6a,Grand Theft Auto V,10,DareJedi,698,9191
3,5c09771d933dea68aaeebb6b,Grand Theft Auto V,4,MaxBenevolent,698,22207
4,5c09771d933dea68aaeebb6c,Grand Theft Auto V,3,BVGNOfficial,698,37691


In [4]:
spark_df=spark.createDataFrame(df[['game_number', 'user_number', 'score']])

In [5]:
spark_df.show()

+-----------+-----------+-----+
|game_number|user_number|score|
+-----------+-----------+-----+
|        698|      18776|    8|
|        698|       2369|    7|
|        698|       9191|   10|
|        698|      22207|    4|
|        698|      37691|    3|
|        698|       2024|    2|
|        698|      36719|    7|
|        698|      31522|    5|
|        698|       7008|    7|
|        698|      39936|    7|
|        698|      40272|   10|
|        698|       2844|   10|
|        698|       1281|   10|
|        698|      25589|    8|
|        698|      36328|    8|
|        698|      33219|    8|
|        698|       5031|   10|
|        698|      24195|    8|
|        698|       1608|   10|
|        698|      16248|   10|
+-----------+-----------+-----+
only showing top 20 rows



In [358]:
(training, test) = spark_df.randomSplit([0.8, 0.2])

In [371]:
#(training, test) = spark_df.randomSplit([0.8, 0.2])

als = ALS(maxIter=20, 
          regParam=0.75, 
          rank=100,
          userCol="user_number",
          itemCol="game_number",
          ratingCol="score",
          coldStartStrategy="drop")

In [372]:
model = als.fit(training)

In [373]:
predictions = model.transform(test)

In [374]:
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="score",
                                predictionCol="prediction")

In [375]:
rmse = evaluator.evaluate(predictions)

In [376]:
rmse

2.9458647186856104

In [421]:
user_factors = model.userFactors.toPandas()
item_factors = model.itemFactors.toPandas()

In [422]:
sample = {'Destiny 2: Forsaken': 8,
         "Everybody's Golf": 8,
         "XCOM 2": 9,
         "NBA 2K17": 10,
         "Marvel's Spider-Man":6}

In [423]:
sample_ids=[]
for key in sample:
    sample_ids.append(int(df[df['game_id']==key].game_number.unique()))

In [424]:
sample_ids

[472, 60, 49, 216, 116]

In [436]:
#item_factors[item_factors['id']==472]['features'].tolist()[0]

In [451]:
item_factors_arr=np.array(item_factors['features'].tolist())

In [455]:
item_factors_arr[0]

array([-0.00172996, -0.3201465 , -0.069617  , -0.21138753, -0.23682658,
       -0.29458925,  0.5124892 ,  0.04235182, -0.04884242,  0.07657851,
        0.35035074,  0.0183627 , -0.03155093,  0.07250211,  0.11819259,
        0.05018757, -0.04044857, -0.1220509 , -0.12729846, -0.30617869,
       -0.35290083,  0.63772959,  0.47556904, -0.18920468,  0.00752726,
       -0.20203499, -0.0945861 ,  0.18771589, -0.17250931,  0.1306888 ,
       -0.34391859, -0.11467288, -0.24894902,  0.25774318,  0.36460879,
        0.47830895,  0.35905328,  0.28451625,  0.30261901, -0.22621883,
        0.15497528, -0.00191984,  0.14159556,  0.03377046,  0.31136963,
        0.2107705 , -0.37906101, -0.0177942 ,  0.0665674 , -0.00548473,
        0.19559778, -0.16504686,  0.15123864, -0.10949144,  0.08185766,
        0.19201919,  0.13711433, -0.09253408, -0.15355504, -0.32237515,
        0.02699889, -0.30889875,  0.33711123, -0.27634889, -0.3053216 ,
       -0.54909909,  0.37919599, -0.28650054, -0.02761799,  0.03

In [463]:
num_games=len(df['game_number'].unique().tolist())

In [461]:
sample_array=np.zeros(num_games)
for i in range(num_games):
    

In [462]:
sample_array.shape

(1587,)

In [467]:
model.

AttributeError: 'Param' object has no attribute 'show'